In [ ]:
#pip install azure-eventhub

In [ ]:
#pip install azure-eventhub-checkpointstoreblob-aio

In [10]:
import time
from datetime import datetime

t = time.time()

print(t)

1628094737.8189394


In [11]:
datetime.fromtimestamp(t)

datetime.datetime(2021, 8, 4, 9, 32, 17, 818939)

In [16]:
print(datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S"))

2021-08-04 09:32:17


In [14]:
datetime.utcnow()

datetime.datetime(2021, 8, 4, 16, 34, 22, 927480)

In [17]:
print(datetime.utcnow().isoformat(sep=' ', timespec='milliseconds'))

2021-08-04 16:35:25.457


In [4]:
disconnect()

# Code chunk below pushes live E4 data to Azure

In [4]:
## Successful to recieve data from E4 and simultaneously push it to Azure

import socket
import time
import pylsl
from datetime import datetime 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData
import os
import time

from random import randint
%matplotlib qt
#%matplotlib inline


participant = input("Participant's Full Name: ")
participant_ID = input("Participant ID: ")
session_ID = participant_ID + str(time.time())

# SELECT DATA TO STREAM
acc = True      # 3-axis acceleration
bvp = True     # Blood Volume Pulse
gsr = True      # Galvanic Skin Response (Electrodermal Activity)
tmp = True      # Temperature
ibi = True      # Interbeat Interval and Heartbeat
bat = False      # Device Battery
tag = False     # Tag taken from the device (by pressing the button)

serverAddress = '127.0.0.1'
serverPort = 28000
bufferSize = 4096

deviceID = 'CDCB11' # 'A03003'

def connect():
    global s
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(3)

    print("Connecting to server")
    s.connect((serverAddress, serverPort))
    print("Connected to server\n")

    print("Devices available:")
    s.send("device_list\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Connecting to device")
    s.send(("device_connect " + deviceID + "\r\n").encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Pausing data receiving")
    s.send("pause ON\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
connect()

time.sleep(1)

def suscribe_to_data():
    if acc:
        print("Suscribing to ACC")
        s.send(("device_subscribe " + 'acc' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bvp:
        print("Suscribing to BVP")
        s.send(("device_subscribe " + 'bvp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if gsr:
        print("Suscribing to GSR")
        s.send(("device_subscribe " + 'gsr' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tmp:
        print("Suscribing to Temp")
        s.send(("device_subscribe " + 'tmp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if ibi:
        print("Suscribing to IBI")
        s.send(("device_subscribe " + 'ibi' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bat:
        print("Suscribing to Batt")
        s.send(("device_subscribe " + 'bat' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tag:
        print("Suscribing to Tag")
        s.send(("device_subscribe " + 'tag' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))

    print("Resuming data receiving")
    s.send("pause OFF\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
suscribe_to_data()

time.sleep(1)

def reconnect():
    print("Reconnecting...")
    connect()
    suscribe_to_data()
    #stream()

def disconnect():
    s.send("device_disconnect\r\n".encode())

def convert_to_json(sample):
    stream_type = sample.split()[0]
    if stream_type == "E4_Acc":
        new_timestamp = datetime.fromtimestamp(float(sample.split()[1])).isoformat(sep=' ', timespec='milliseconds') #recent update to include milliseconds
        new_timestamp_unix = sample.split()[1] #recent update to include unix timestamp to make querying easier
        new_data = [int(samples[i].split()[2].replace(',','.')), int(samples[i].split()[3].replace(',','.')), int(samples[i].split()[4].replace(',','.'))]
        sample_jsonX = {"stream_type" : "E4_Acc_X", "Value" : new_data[0], "dateTime" : new_timestamp, "dateTime_Unix" : new_timestamp_unix, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_jsonY = {"stream_type" : "E4_Acc_Y", "Value" : new_data[1], "dateTime" : new_timestamp, "dateTime_Unix" : new_timestamp_unix, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_jsonZ = {"stream_type" : "E4_Acc_Z", "Value" : new_data[2], "dateTime" : new_timestamp, "dateTime_Unix" : new_timestamp_unix, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_json = [sample_jsonX, sample_jsonY, sample_jsonZ]
        sample_json = json.dumps(sample_json)    #might have issues from trying to batch more than 1 json at a time later in the code
    else:
        new_timestamp = datetime.fromtimestamp(float(sample.split()[1])).isoformat(sep=' ', timespec='milliseconds') #recent update to include milliseconds
        new_timestamp_unix = sample.split()[1] #recent update to include unix timestamp to make querying easier
        new_data = float(sample.split()[2])
        sample_json = {"stream_type" : stream_type, "Value" : new_data, "dateTime" : new_timestamp, "dateTime_Unix" : new_timestamp_unix, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_json = json.dumps(sample_json)
    return(sample_json)

eventhub_name = 'eventhub'
client = EventHubProducerClient.from_connection_string(os.environ.get('CONNECTION_STRING'), eventhub_name=eventhub_name)

try:
    while True:
        response = s.recv(bufferSize).decode("utf-8")
        #print(response)
        if "connection lost to device" in response:
            print(response.decode("utf-8"))
            reconnect()
        event_data_batch = client.create_batch()
        samples = response.split("\r\n")
        for i in range(len(samples)-1):
            sample_json = convert_to_json(samples[i])
            print(sample_json)
            event_data_batch.add(EventData(sample_json))
        client.send_batch(event_data_batch)
except KeyboardInterrupt:
    print('interrupted!')
    disconnect()



Connecting to server
Connected to server

Devices available:
R device_list 1 | CDCB11 Empatica_E4 

Connecting to device
R device_connect OK

Pausing data receiving
R pause ON

Suscribing to ACC
R device_subscribe acc OK

Suscribing to BVP
R device_subscribe bvp OK

Suscribing to GSR
R device_subscribe gsr OK

Suscribing to Temp
R device_subscribe tmp OK

Suscribing to IBI
R device_subscribe ibi OK

Resuming data receiving
R pause OFF



Can't call on_state_changed during garbage collection, please be sure to close or use a context manager
Can't call on_connection_state_changed during garbage collection


{"stream_type": "E4_Bvp", "Value": 90.59103, "dateTime": "2021-08-10 10:06:01.770", "dateTime_Unix": "1628615161.77046", "Session_ID": "0011628615160.396448", "Participant": "Tony Banks", "Participant_ID": "001"}
{"stream_type": "E4_Bvp", "Value": 101.02, "dateTime": "2021-08-10 10:06:01.786", "dateTime_Unix": "1628615161.78608", "Session_ID": "0011628615160.396448", "Participant": "Tony Banks", "Participant_ID": "001"}
{"stream_type": "E4_Bvp", "Value": 109.3755, "dateTime": "2021-08-10 10:06:01.801", "dateTime_Unix": "1628615161.80171", "Session_ID": "0011628615160.396448", "Participant": "Tony Banks", "Participant_ID": "001"}
{"stream_type": "E4_Bvp", "Value": 112.5696, "dateTime": "2021-08-10 10:06:01.817", "dateTime_Unix": "1628615161.81733", "Session_ID": "0011628615160.396448", "Participant": "Tony Banks", "Participant_ID": "001"}
{"stream_type": "E4_Bvp", "Value": 107.7316, "dateTime": "2021-08-10 10:06:01.832", "dateTime_Unix": "1628615161.83296", "Session_ID": "0011628615160.

In [3]:
disconnect()

In [1]:
import os

os.environ['CONNECTION_STRING'] = 'Endpoint=sb://pythoneventhb.servicebus.windows.net/;SharedAccessKeyName=eventpolicy;SharedAccessKey=gR5uUifVoEXQpY57GvQMvZ/YUpAqA87A/hSo3JgdgC8=;EntityPath=eventhub'